In [7]:
!pip install fastapi uvicorn wandb pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.5 MB/s eta 0:00:00


In [8]:
!ngrok authtoken 2mdjIHkDVwk2IvhPOEvsHw6Phjm_MwNg7npSHD92NZxn2MVz

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
os.environ["WANDB_API_KEY"] = "ddb79e032887f0a8e6e54fc954e8fdc8a40c1af2"

In [58]:
%%writefile app.py

import os
import wandb
import sklearn
from joblib import load
from fastapi import FastAPI, HTTPException
from contextlib import asynccontextmanager
from pydantic import BaseModel
import wandb
import joblib
import pandas as pd
import numpy as np

app = FastAPI()

# Define input data schema
class PredictionRequest(BaseModel):
    age: int
    KM_Driven: int
    Fuel_Type: str
    Transmission: str
    Owner_Type: str
    Seats: int
    make: str
    mileage: int
    engine: int
    power: int
    Location: str

# Load the model from WandB
def load_model_from_wandb(project_name: str, artifact_name: str):
    try:
        # Initialize WandB
        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        model_path = artifact.file()  # Assumes a single model file in the artifact
        model = joblib.load(model_path)
        run.finish()  # End the WandB run
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")

project_name = "mlops_usedcar"  # Replace with your project name
artifact_name = "Linear_Model_UsedCar:latest"  # Replace with your artifact name
ml_model = load_model_from_wandb(project_name, artifact_name)

# Prediction endpoint
@app.post("/predict")
def predict(input_data: PredictionRequest):
    try:
        # Convert input data to a dictionary for prediction
        input_dict = input_data.dict()

        df = pd.DataFrame(input_dict, index = [0])

        # Call the model's prediction method
        prediction = ml_model.predict(df)

        # Return the prediction result
        return {f"Estimated Car Price: INR {np.round(prediction[0], 2)} Lakhs"}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {e}")


Overwriting app.py


In [60]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 &

nohup: appending output to 'nohup.out'


In [56]:
!ps -ax | grep uvicorn

  11654 ?        Rl     0:02 /usr/bin/python3 /usr/local/bin/uvicorn app:app --host 0.0.0.0 --port 8
  11675 ?        S      0:00 /bin/bash -c ps -ax | grep uvicorn
  11677 ?        S      0:00 grep uvicorn


In [57]:
from pyngrok import ngrok

# Expose the FastAPI app
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://95e4-34-139-99-78.ngrok-free.app" -> "http://localhost:8000"
